Este módulo obtiene N cantidad de cajeros de cualquier tipo de los 4 pre-cluster definidos.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../../Insumos/df_general.parquet')

In [2]:
def construir_resumen_estructural(df):
    
    resumen = (
        df
        .groupby("cajero")
        .agg(
            dias_obs=("retiro", "count"),
            mediana=("retiro", "median"),
            p95=("retiro", lambda x: x.quantile(0.95))
        )
        .reset_index()
    )
    
    resumen["ratio_p95_mediana"] = (
        resumen["p95"] / resumen["mediana"]
    )
    
    resumen["ratio_p95_mediana"] = resumen["ratio_p95_mediana"].replace([np.inf, -np.inf], 0)
    resumen["ratio_p95_mediana"] = resumen["ratio_p95_mediana"].fillna(0)
    
    return resumen

In [3]:
def aplicar_reglas_precluster(
    df_resumen,
    umbral_grande,
    umbral_estable,
    umbral_evento,
    umbral_picos
):
    
    def clasificar(row):
        mediana = row["mediana"]
        ratio = row["ratio_p95_mediana"]
        
        if mediana == 0:
            return "event_driven"
        elif (mediana >= umbral_grande) and (ratio <= umbral_estable):
            return "grande_y_estable"
        elif ratio >= umbral_evento:
            return "event_driven"
        elif ratio >= umbral_picos:
            return "normal_con_picos"
        else:
            return "normal_estable"
    
    df_resumen["etiqueta"] = df_resumen.apply(clasificar, axis=1)
    
    return df_resumen

In [4]:
resumen = construir_resumen_estructural(df)

df_clasificados = aplicar_reglas_precluster(
    resumen,
    umbral_grande=200000,
    umbral_estable=3,
    umbral_evento=8,
    umbral_picos=5
)

In [5]:
def seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster,
    n,
    modo="aleatorio",
    random_state=42
):
    
    df_cluster = df_clasificados[
        df_clasificados["etiqueta"] == cluster
    ].copy()
    
    total_disponibles = len(df_cluster)
    
    if total_disponibles == 0:
        raise ValueError(f"No existen cajeros en el cluster '{cluster}'")
    
    if n > total_disponibles:
        print(f"Solo hay {total_disponibles} cajeros en '{cluster}'. Se devolverán todos.")
        n = total_disponibles
    
    if modo == "aleatorio":
        seleccion = df_cluster.sample(n=n, random_state=random_state)
        
    elif modo == "mayor_ratio":
        seleccion = df_cluster.sort_values(
            "ratio_p95_mediana",
            ascending=False
        ).head(n)
        
    elif modo == "mayor_mediana":
        seleccion = df_cluster.sort_values(
            "mediana",
            ascending=False
        ).head(n)
        
    else:
        raise ValueError("Modo no válido.")
    
    return seleccion.reset_index(drop=True)

In [6]:
'''Las 4 categorías que existen son:

    - event_driven
    - grande_y_estable
    - normal_con_picos
    - normal_estable
'''
seleccionar_cajeros_por_cluster(
    df_clasificados,
    cluster="event_driven",
    n=5,
    modo="mayor_ratio"
)

#NOTA: Sería mejor si la columna etiqueta estuviera junto a cajero. 

,cajero,dias_obs,mediana,p95,ratio_p95_mediana,etiqueta
0,JF001931,754,50.0,108760.0,2175.200000,event_driven
1,JF000995,754,350.0,209530.0,598.657143,event_driven
2,JF001356,754,1600.0,329355.0,205.846875,event_driven
3,JF002402,754,7850.0,862425.0,109.863057,event_driven
4,JF001124,754,3050.0,170250.0,55.819672,event_driven


# Core del Extractor:

In [ ]:
def seleccionar_cajeros_multi_cluster(
    df_clasificados,
    solicitud_dict,
    modo="aleatorio",
    random_state=42
):
    """
    solicitud_dict ejemplo:
    {
        "normal_estable": 50,
        "event_driven": 30,
        "normal_con_picos": 10
    }
    """
    
    resultados = {}
    resumen_rows = []
    
    total_solicitado = sum(solicitud_dict.values())
    
    for cluster, n in solicitud_dict.items():
        
        df_cluster = df_clasificados[
            df_clasificados["etiqueta"] == cluster
        ].copy()
        
        disponibles = len(df_cluster)
        
        if disponibles == 0:
            print(f"No existen cajeros en {cluster}. Se omite.")
            continue
        
        if n > disponibles:
            print(f"{cluster}: solo hay {disponibles}. Se ajusta N.")
            n = disponibles
        
        # Selección
        if modo == "aleatorio":
            seleccion = df_cluster.sample(n=n, random_state=random_state)
        elif modo == "mayor_ratio":
            seleccion = df_cluster.sort_values(
                "ratio_p95_mediana",
                ascending=False
            ).head(n)
        elif modo == "mayor_mediana":
            seleccion = df_cluster.sort_values(
                "mediana",
                ascending=False
            ).head(n)
        else:
            raise ValueError("Modo no válido.")
        
        lista_cajeros = seleccion["cajero"].tolist()
        
        resultados[cluster] = lista_cajeros
        
        porcentaje = (n / total_solicitado) * 100
        
        resumen_rows.append({
            "Pre-cluster": cluster.upper(),
            "Porcentaje (%)": round(porcentaje, 2), #establecer si el parámetro es verdaderamente funcional.
            "Número (n)": n
        })
    
    resumen_df = pd.DataFrame(resumen_rows)
    
    return resultados, resumen_df

## Ejemplo de uso

In [8]:
solicitud = {
    "normal_estable": 40,
    "event_driven": 20,
    "normal_con_picos": 15,
    "grande_y_estable": 5
}

listas_atm, tabla_resumen = seleccionar_cajeros_multi_cluster(
    df_clasificados,
    solicitud,
    modo="aleatorio"
)

In [ ]:
tabla_resumen

,Pre-cluster,Porcentaje (%),Número (n)
0,NORMAL_ESTABLE,50.00,40
1,EVENT_DRIVEN,25.00,20
2,NORMAL_CON_PICOS,18.75,15
3,GRANDE_Y_ESTABLE,6.25,5


In [ ]:
#diccionario que contiene todos los elementos solicitados
listas_atm

{'normal_estable': ['JF002100',
  'JF001496',
  'JF002182',
  'JF001025',
  'JF000990',
  'JF002865',
  'JF001006',
  'JF000565',
  'JF000963',
  'JF000140',
  'JF001244',
  'JF002886',
  'JF002827',
  'JF001434',
  'JF001879',
  'JF002129',
  'JF000727',
  'JF001571',
  'JF000065',
  'JF000068',
  'JF002903',
  'JF002939',
  'JF001014',
  'JF002861',
  'JF001973',
  'JF001483',
  'JF002013',
  'JF000673',
  'JF001300',
  'JF002017',
  'JF000131',
  'JF002266',
  'JF001294',
  'JF001511',
  'JF001638',
  'JF001298',
  'JF002193',
  'JF001258',
  'JF000972',
  'JF000050'],
 'event_driven': ['JF000408',
  'JF001695',
  'JF001201',
  'JF001762',
  'JF002238',
  'JF001892',
  'JF002419',
  'JF000805',
  'JF002083',
  'JF002780',
  'JF002131',
  'JF999988',
  'JF002604',
  'JF001225',
  'JF002207',
  'JF002189',
  'JF002798',
  'JF002978',
  'JF001508',
  'JF000985'],
 'normal_con_picos': ['JF000195',
  'JF001510',
  'JF000604',
  'JF000675',
  'JF000126',
  'JF002841',
  'JF001595',
  'JF0

In [16]:
print(listas_atm.get('normal_estable'))
print(listas_atm.get('event_driven'))
print(listas_atm.get('normal_con_picos'))
print(listas_atm.get('grande_y_estable'))

['JF002100', 'JF001496', 'JF002182', 'JF001025', 'JF000990', 'JF002865', 'JF001006', 'JF000565', 'JF000963', 'JF000140', 'JF001244', 'JF002886', 'JF002827', 'JF001434', 'JF001879', 'JF002129', 'JF000727', 'JF001571', 'JF000065', 'JF000068', 'JF002903', 'JF002939', 'JF001014', 'JF002861', 'JF001973', 'JF001483', 'JF002013', 'JF000673', 'JF001300', 'JF002017', 'JF000131', 'JF002266', 'JF001294', 'JF001511', 'JF001638', 'JF001298', 'JF002193', 'JF001258', 'JF000972', 'JF000050']
['JF000408', 'JF001695', 'JF001201', 'JF001762', 'JF002238', 'JF001892', 'JF002419', 'JF000805', 'JF002083', 'JF002780', 'JF002131', 'JF999988', 'JF002604', 'JF001225', 'JF002207', 'JF002189', 'JF002798', 'JF002978', 'JF001508', 'JF000985']
['JF000195', 'JF001510', 'JF000604', 'JF000675', 'JF000126', 'JF002841', 'JF001595', 'JF000281', 'JF000198', 'JF000229', 'JF002345', 'JF001470', 'JF000469', 'JF001400', 'JF001071']
['JF002764', 'JF002974', 'JF001718', 'JF002310', 'JF000125']
